In [13]:
import duckdb

In [14]:
test_suites_dir = "../test-suites/duckdb/"
file_path = test_suites_dir + "aggregate/aggregates/string_agg_union.test"

In [15]:
"""
statement ok/error
query <type-string> <sort-mode> <label>
actual query here
---
expected result
"""

'\nstatement ok/error\nquery <type-string> <sort-mode> <label>\nactual query here\n---\nexpected result\n'

In [16]:
def parse_test_file(filename):
    queries = []
    current_query = None
    in_query_block, collecting_result = False, False
    current_result_lines = []

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('#'):
                continue
            
            #empty lines
            if not line:
                if in_query_block:
                    # If we were collecting result lines, join and save them
                    if collecting_result and current_query:
                        current_query["result"] = "\n".join(current_result_lines).strip()
                        current_result_lines = []
                        collecting_result = False
                    queries.append(current_query)
                    current_query = None
                    in_query_block = False
                continue
            
            # Check for start of a new query block
            if not in_query_block:
                current_query = {
                    "success": None,
                    "query": "",
                    "result_specs": {
                        "nr_of_columns": None,
                        "sort": None,
                        "label": None
                    },
                    "result": None
                }
                in_query_block = True
            
            # Check for expected execution results (statement ok / statement fail)
            if line.startswith("statement"):
                current_query["success"] = line.endswith("ok")
                continue
            
            # Check for expected columns (query I / query II)
            if line.startswith("query"):
                result_specs = line.split(" ")
                if len(result_specs) >= 2:
                    current_query['result_specs']['nr_of_columns'] = len(result_specs[1])
                if len(result_specs) >= 3:
                    current_query['result_specs']['sort'] = result_specs[2]
                if len(result_specs) >= 4:
                    current_query['result_specs']['label'] = result_specs[3]
                continue
           
            
            # Check for expected result lines (after ----)
            if line.startswith("----"):
                collecting_result = True
                continue
            
            # If we are in the result collection mode, accumulate result lines
            if collecting_result:
                current_result_lines.append(line)
                continue
            
            # Accumulate query lines
            current_query["query"] += line + " "
        
        # Append the last query block if there's any
        if current_query:
            queries.append(current_query)

    return queries


In [24]:
queries = parse_test_file(file_path)

In [25]:
for query in queries:
    duckdb.sql(query['query']).show()

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

┌─────────────────────┐
│    my_string_agg    │
│       varchar       │
├─────────────────────┤
│ text1, text1, text1 │
└─────────────────────┘

┌─────────────────────┐
│    my_string_agg    │
│       varchar       │
├─────────────────────┤
│ text1, text1, text1 │
└─────────────────────┘

